In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import scipy.io as sio
from dataclasses import dataclass
from typing import List, Tuple
import os
from dotenv import load_dotenv
load_dotenv()
import tidy3d as td
from tidy3d import web
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import AutomationModule as AM
from scipy.optimize import curve_fit
from natsort import natsorted
import numpy as np
import matplotlib.animation as animation
import xarray as xr
import imageio
import matplotlib
import gc

tidy3dAPI = os.environ["API_TIDY3D_KEY"]

In [ ]:
import AutomationModule as AM
#structure_1 = AM.loadFromFile(key = tidy3dAPI, file_path="data/chi 0.3 N1000 Sample Beam Spreading Larger Space Absorber Conditions 2/z_incidence/pnas_d_t_Localization 0.12 - 0.15 - Sample_0.txt")
#structure_1 = AM.loadFromFile(key = tidy3dAPI, file_path="data/chi 0.3 N1000 Sample Beam Spreading Larger Space Absorber Conditions 2/z_incidence/pnas_d_t_Localization 0.28 - 0.31 - Sample_0.txt")
structure_1 = AM.loadFromFile(key = tidy3dAPI, file_path="data/chi 0.3 N1000 Sample Beam Spreading Larger Space Absorber Conditions/z_incidence/pnas_d_t_Localization 0.28 - 0.31 - Sample_0.txt")

In [ ]:
structure_1.sim_data.simulation.sources

In [ ]:
def create_movie(field_time_out, monitor_lambdas,name='',type='t',log=False,path="",frames_per_second=1,rem_frames=False, normalize=True):
    frames = []

    max_values = field_time_out.max(axis=(0,1,2)) if normalize else 1

    field_time_out = field_time_out/max_values
   

    if log:
        field_log = np.log10((field_time_out))
        folder_pics = "logPics"
        
    else:
        field_log = ((field_time_out))
        folder_pics = "linPics"

    if not os.path.exists(f'{path}/{folder_pics}'):
            os.makedirs(f'{path}/{folder_pics}')
            print(f"Folder {path}/{folder_pics} created successfully.")

    for i, time in enumerate(field_time_out.t):
        if os.path.isfile(f'{path}/{folder_pics}/frame_{i}.png'):
            frames.append(f'{path}/{folder_pics}/frame_{i}.png')
            continue
        try:
            fig, ax = plt.subplots(figsize=(14/2, 18/2))
            if type=="t":
                pcolormesh = (field_log).isel(t=i).squeeze().plot.pcolormesh(ax=ax,cmap="plasma")
            else:
                pcolormesh = (field_log).isel(f=i).squeeze().plot.pcolormesh(ax=ax,cmap="plasma")

            ax.set_aspect('auto', adjustable='box')
            plt.ylim(-50,50)
            try:
                plt.title(f'Time: {str(np.array(field_time_out['t'][()][i])*1e12)} ps')
            except:
                plt.title(f'$\\nu$: {(1/np.array(td.C_0/field_time_out['f'][()][i])):.4g}')


            # Save the frame
            plt.savefig(f'{path}/{folder_pics}/frame_{i}.png')
            plt.close(fig)
            frames.append(f'{path}/{folder_pics}/frame_{i}.png')
        except:
            break

        fig.clf()
        plt.close()
        del pcolormesh
        gc.collect()
        

    name_movie = f'{path}/{name}.mp4' if name else f'output/anderson/d(t) analysis/Diameter d(t) at output of the structure Range - {monitor_lambdas[0]:.3g} - {monitor_lambdas[-1]:.3g}.mp4'
    with imageio.get_writer(name_movie, fps=frames_per_second) as writer:
        for frame in frames:
            image = imageio.imread(frame)
            writer.append_data(image)

    # Optionally, remove the individual frames if no longer needed
    if rem_frames:
        for frame in frames:
            os.remove(frame)
    
    return False

In [ ]:
a=1
#lambdas =  a/np.array([0.12,0.15])
lambdas =  a/np.array([0.28,0.31])

In [ ]:

field_time_out = structure_1.sim_data.get_intensity("time_monitorFieldOut")
del structure_1


In [ ]:
font = {
        'weight' : 'bold',
        'size'   : 6}

matplotlib.rc('font', **font)
create_movie(field_time_out,a/lambdas,name=f"timemonitor {a/lambdas[0]:.3g} - {a/lambdas[-1]:.3g}",type="t",log=False,path=f"output/2D SHU/Beam Spreading Chi 0.3 N10000",frames_per_second=10)

In [ ]:
create_movie(field_time_out,a/lambdas,name=f"timemonitorlog {a/lambdas[0]:.3g} - {a/lambdas[-1]:.3g}",type="t",log=True,path=f"output/2D SHU/Beam Spreading Chi 0.3 N10000",frames_per_second=10)
